In [1]:
import pandas as pd
import requests
import json
import matplotlib
%matplotlib inline   
# whenever i want to display matplotlib, i need to type this code in order to display the plot

In [2]:
city_code= {"cancun": 34713,"buenos aires":19264,"rio de janeiro": 24146}

In [22]:
str(city_code["CanCun".lower()])

'34713'

In [18]:
dep = "2019-12-21"
days =7
from datetime import datetime
from datetime import timedelta
dep_new =datetime.strptime(dep, '%Y-%m-%d').date()
arr= dep_new + timedelta(days=(days-1))

arr_string = arr.strftime("%Y-%m-%d")

arr_string

'2019-12-27'

In [ ]:
# It is imperative to get the desired city codes from https://rapidapi.com/apidojo/api/kajak?endpoint=5c131371e4b09c6b17cfa632
city_code= {"cancun": 34713,"buenos aires":19264,"rio de janeiro": 24146}


def Hotel_booking_function(city, arrival_date, days=7, stars=4):
    
    
    # Calculate the date of departure based on departure date and no of days staying there
    from datetime import datetime
    from datetime import timedelta
    dep = datetime.strptime(arrival_date, "%Y-%m-%d").date()+ timedelta(days=(days-1))
    departure_date = dep.strftime("%Y-%m-%d")
    
    # Create the API for the search
    url = "

In [38]:
# It is imperative to get the desired city codes from https://rapidapi.com/apidojo/api/kajak?endpoint=5c131371e4b09c6b17cfa632
city_code= {"cancun": 34713,"buenos aires":19264,"rio de janeiro": 24146}


def Hotel_booking_function(city, arrival_date, days=7, stars=4):
    
    
    # Calculate the date of departure based on departure date and no of days staying there
    from datetime import datetime
    from datetime import timedelta
    dep = datetime.strptime(arrival_date, "%Y-%m-%d").date()+ timedelta(days=(days-1))
    departure_date = dep.strftime("%Y-%m-%d")
    
    # Create the API for the search
    url = "https://apidojo-kayak-v1.p.rapidapi.com/hotels/create-session"

    querystring = {"currency":"EUR","rooms":"1","citycode":str(city_code[city.lower()]),"checkin":arrival_date,"checkout":departure_date,"adults":"2"}

    headers = {
        'x-rapidapi-host': "apidojo-kayak-v1.p.rapidapi.com",
        'x-rapidapi-key': "b0b414424fmsh520700b66579ef2p149f78jsn1233a2112518"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    # Create a dataframe for this particular holiday and time selection
    # Filter the dataframe and rename the columns
    hotel_raw_list = pd.DataFrame(response.json()["hotelset"])
    hotel_consolidated_list= hotel_raw_list[["name","price","userrating","stars","cheapestProviderName","address","phone","city","distance"]]
    hotel_consolidated_list = hotel_consolidated_list.rename(str.capitalize, axis='columns').rename(columns={"Name": "Hotel Name","Userrating":"User Rating", "Cheapestprovidername": "Cheapest Provider Name", "Price":"Hotel Price per Night", "Distance":"Distance from City Centre"})
    
    #Convert price from str to float
    hotel_consolidated_list["Hotel Price per Night"]= hotel_consolidated_list["Hotel Price per Night"].replace("\u20AC","", regex =True).astype(float)
    
    #Add a "id" column which fits the following format: "city_arrival date" e.g. "cancun_2019-12-21"
    hotel_consolidated_list["id"] = city+"_"+arrival_date
    
    #Find the cheapest hotel for the city given the stars rating
    list_filtered_with_stars= hotel_consolidated_list[hotel_consolidated_list["Stars"]==stars]
    cheapest_hotel= list_filtered_with_stars[list_filtered_with_stars["Hotel Price per Night"] == list_filtered_with_stars["Hotel Price per Night"].min()].reset_index(drop= True)
    return cheapest_hotel

In [39]:
#Testing the function
Hotel_booking_function("Cancun", "2019-12-21")

,Hotel Name,Hotel Price per Night,User Rating,Stars,Cheapest Provider Name,Address,Phone,City,Distance from City Centre,id
0,Krystal Urban Cancun,80.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,Cancun_2019-12-21


In [44]:
cities = ["cancun", "buenos aires", "rio de janeiro"]
date= ["2019-12-21","2020-01-18","2020-02-22"]

In [61]:
compiled_data= pd.DataFrame()
for city in cities:
    for day in date:
        data= Hotel_booking_function(city, day)
        compiled_data= compiled_data.append(data)


compiled_data

,Hotel Name,Hotel Price per Night,User Rating,Stars,Cheapest Provider Name,Address,Phone,City,Distance from City Centre,id
0,Krystal Urban Cancun,80.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2019-12-21
0,Hotel Kavia,30.0,81,4.0,SnapTravel,Claveles 109,+52 998 217 7742,Cancún,0.502080,cancun_2020-01-18
0,Krystal Urban Cancun,56.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2020-02-22
0,El Conquistador Hotel,36.0,79,4.0,Prestigia,Suipacha 948,+54 114 328 3012,Buenos Aires,1.794185,buenos aires_2019-12-21
1,Unique Palacio San Telmo,36.0,77,4.0,Priceline,Moreno 606,+54 114 807 7008,Buenos Aires,0.282323,buenos aires_2019-12-21
0,Cuatro Reyes,22.0,69,4.0,SnapTravel,Av Independencia 1391,+54 114 384 8000,Buenos Aires,0.900278,buenos aires_2020-01-18
0,Wilton Hotel Buenos Aires,61.0,79,4.0,Wilton Hotel Buenos Aires,Avenida Callao 1162,+54 114 811 1818,Buenos Aires,2.473522,buenos aires_2020-02-22
0,Hotel Atlântico Travel,33.0,81,4.0,SnapTravel,Rua Barata Ribeiro Nº 544,+55 213 479 2000,Rio de Janeiro,7.798410,rio de janeiro_2019-12-21
1,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2019-12-21
0,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2020-01-18


In [62]:
compiled_data.reset_index(drop=True)

,Hotel Name,Hotel Price per Night,User Rating,Stars,Cheapest Provider Name,Address,Phone,City,Distance from City Centre,id
0,Krystal Urban Cancun,80.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2019-12-21
1,Hotel Kavia,30.0,81,4.0,SnapTravel,Claveles 109,+52 998 217 7742,Cancún,0.502080,cancun_2020-01-18
2,Krystal Urban Cancun,56.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2020-02-22
3,El Conquistador Hotel,36.0,79,4.0,Prestigia,Suipacha 948,+54 114 328 3012,Buenos Aires,1.794185,buenos aires_2019-12-21
4,Unique Palacio San Telmo,36.0,77,4.0,Priceline,Moreno 606,+54 114 807 7008,Buenos Aires,0.282323,buenos aires_2019-12-21
5,Cuatro Reyes,22.0,69,4.0,SnapTravel,Av Independencia 1391,+54 114 384 8000,Buenos Aires,0.900278,buenos aires_2020-01-18
6,Wilton Hotel Buenos Aires,61.0,79,4.0,Wilton Hotel Buenos Aires,Avenida Callao 1162,+54 114 811 1818,Buenos Aires,2.473522,buenos aires_2020-02-22
7,Hotel Atlântico Travel,33.0,81,4.0,SnapTravel,Rua Barata Ribeiro Nº 544,+55 213 479 2000,Rio de Janeiro,7.798410,rio de janeiro_2019-12-21
8,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2019-12-21
9,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2020-01-18


In [66]:
compiled_data1= compiled_data.reset_index(drop =True)

In [67]:
compiled_data1

,Hotel Name,Hotel Price per Night,User Rating,Stars,Cheapest Provider Name,Address,Phone,City,Distance from City Centre,id
0,Krystal Urban Cancun,80.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2019-12-21
1,Hotel Kavia,30.0,81,4.0,SnapTravel,Claveles 109,+52 998 217 7742,Cancún,0.502080,cancun_2020-01-18
2,Krystal Urban Cancun,56.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2020-02-22
3,El Conquistador Hotel,36.0,79,4.0,Prestigia,Suipacha 948,+54 114 328 3012,Buenos Aires,1.794185,buenos aires_2019-12-21
4,Unique Palacio San Telmo,36.0,77,4.0,Priceline,Moreno 606,+54 114 807 7008,Buenos Aires,0.282323,buenos aires_2019-12-21
5,Cuatro Reyes,22.0,69,4.0,SnapTravel,Av Independencia 1391,+54 114 384 8000,Buenos Aires,0.900278,buenos aires_2020-01-18
6,Wilton Hotel Buenos Aires,61.0,79,4.0,Wilton Hotel Buenos Aires,Avenida Callao 1162,+54 114 811 1818,Buenos Aires,2.473522,buenos aires_2020-02-22
7,Hotel Atlântico Travel,33.0,81,4.0,SnapTravel,Rua Barata Ribeiro Nº 544,+55 213 479 2000,Rio de Janeiro,7.798410,rio de janeiro_2019-12-21
8,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2019-12-21
9,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2020-01-18


In [71]:
compiled_data1.drop(compiled_data1.index[8])

,Hotel Name,Hotel Price per Night,User Rating,Stars,Cheapest Provider Name,Address,Phone,City,Distance from City Centre,id
0,Krystal Urban Cancun,80.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2019-12-21
1,Hotel Kavia,30.0,81,4.0,SnapTravel,Claveles 109,+52 998 217 7742,Cancún,0.502080,cancun_2020-01-18
2,Krystal Urban Cancun,56.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2020-02-22
3,El Conquistador Hotel,36.0,79,4.0,Prestigia,Suipacha 948,+54 114 328 3012,Buenos Aires,1.794185,buenos aires_2019-12-21
4,Unique Palacio San Telmo,36.0,77,4.0,Priceline,Moreno 606,+54 114 807 7008,Buenos Aires,0.282323,buenos aires_2019-12-21
5,Cuatro Reyes,22.0,69,4.0,SnapTravel,Av Independencia 1391,+54 114 384 8000,Buenos Aires,0.900278,buenos aires_2020-01-18
6,Wilton Hotel Buenos Aires,61.0,79,4.0,Wilton Hotel Buenos Aires,Avenida Callao 1162,+54 114 811 1818,Buenos Aires,2.473522,buenos aires_2020-02-22
7,Hotel Atlântico Travel,33.0,81,4.0,SnapTravel,Rua Barata Ribeiro Nº 544,+55 213 479 2000,Rio de Janeiro,7.798410,rio de janeiro_2019-12-21
9,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2020-01-18
10,Novotel Rj Porto Atlantico,99.0,89,4.0,Novotel,"Avenida Professor Pereira Reis, 49",+55 213 077 2500,Rio de Janeiro,0.791495,rio de janeiro_2020-02-22


In [72]:
compiled_data1= compiled_data1.drop(compiled_data1.index[8])
compiled_data1= compiled_data1.drop(compiled_data1.index[3])

In [74]:
compiled_data1 = compiled_data1.reset_index(drop=True)

In [75]:
compiled_data1

,Hotel Name,Hotel Price per Night,User Rating,Stars,Cheapest Provider Name,Address,Phone,City,Distance from City Centre,id
0,Krystal Urban Cancun,80.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2019-12-21
1,Hotel Kavia,30.0,81,4.0,SnapTravel,Claveles 109,+52 998 217 7742,Cancún,0.502080,cancun_2020-01-18
2,Krystal Urban Cancun,56.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2020-02-22
3,Unique Palacio San Telmo,36.0,77,4.0,Priceline,Moreno 606,+54 114 807 7008,Buenos Aires,0.282323,buenos aires_2019-12-21
4,Cuatro Reyes,22.0,69,4.0,SnapTravel,Av Independencia 1391,+54 114 384 8000,Buenos Aires,0.900278,buenos aires_2020-01-18
5,Wilton Hotel Buenos Aires,61.0,79,4.0,Wilton Hotel Buenos Aires,Avenida Callao 1162,+54 114 811 1818,Buenos Aires,2.473522,buenos aires_2020-02-22
6,Hotel Atlântico Travel,33.0,81,4.0,SnapTravel,Rua Barata Ribeiro Nº 544,+55 213 479 2000,Rio de Janeiro,7.798410,rio de janeiro_2019-12-21
7,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2020-01-18
8,Novotel Rj Porto Atlantico,99.0,89,4.0,Novotel,"Avenida Professor Pereira Reis, 49",+55 213 077 2500,Rio de Janeiro,0.791495,rio de janeiro_2020-02-22


In [76]:
compiled_data1.to_csv("data_thieves_hotel_final.csv", index=False)

In [77]:
pd.read_csv("data_thieves_hotel_final.csv")

,Hotel Name,Hotel Price per Night,User Rating,Stars,Cheapest Provider Name,Address,Phone,City,Distance from City Centre,id
0,Krystal Urban Cancun,80.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2019-12-21
1,Hotel Kavia,30.0,81,4.0,SnapTravel,Claveles 109,+52 998 217 7742,Cancún,0.502080,cancun_2020-01-18
2,Krystal Urban Cancun,56.0,80,4.0,Cancelon,"Avenida Sayil SM4A, Lote 7",+52 998 848 8000,Cancún,1.137819,cancun_2020-02-22
3,Unique Palacio San Telmo,36.0,77,4.0,Priceline,Moreno 606,+54 114 807 7008,Buenos Aires,0.282323,buenos aires_2019-12-21
4,Cuatro Reyes,22.0,69,4.0,SnapTravel,Av Independencia 1391,+54 114 384 8000,Buenos Aires,0.900278,buenos aires_2020-01-18
5,Wilton Hotel Buenos Aires,61.0,79,4.0,Wilton Hotel Buenos Aires,Avenida Callao 1162,+54 114 811 1818,Buenos Aires,2.473522,buenos aires_2020-02-22
6,Hotel Atlântico Travel,33.0,81,4.0,SnapTravel,Rua Barata Ribeiro Nº 544,+55 213 479 2000,Rio de Janeiro,7.798410,rio de janeiro_2019-12-21
7,Arosa Rio Hotel,33.0,85,4.0,SnapTravel,Rua Henrique Valadares 150,+55 213 231 8031,Rio de Janeiro,2.027751,rio de janeiro_2020-01-18
8,Novotel Rj Porto Atlantico,99.0,89,4.0,Novotel,"Avenida Professor Pereira Reis, 49",+55 213 077 2500,Rio de Janeiro,0.791495,rio de janeiro_2020-02-22
